In [2]:
### https://www.kaggle.com/dileep070/heart-disease-prediction-using-logistic-regression
import numpy as np
import pandas as pd 
# classifier models:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
# scaler
from sklearn.preprocessing import StandardScaler
# split
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
# matplot libs
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt 
# to reduction
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
# scoring
from sklearn.metrics import fbeta_score, make_scorer
from sklearn import metrics
from sklearn.metrics import confusion_matrix,accuracy_score,roc_curve,classification_report
# to classification report
from sklearn.model_selection import learning_curve
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

Abaixo, funcoes de plots (ou tentativas rs) de plotar o gráfico. O último que postei era essa primeira funcao. Este comentário será apagado em 24 horas, boa noite.

In [5]:
def print_dim_reduced_data(samples_2d, targets):
    target_ids = range(2)
    colors = 'r', 'g'
    for i, c, label in zip(target_ids, colors, targets):
        plt.scatter(samples_2d[targets == i, 0], samples_2d[targets == i, 1], c=c, label=label)
    plt.grid(linestyle='--')
    plt.legend(bbox_to_anchor=(1.15, 0.725, 0, 0))
    plt.show()
    plt.close()

In [45]:
def print_val_scores(model, X_train, y_train, scoring=['f1_weighted','f1_macro','recall_weighted','precision_weighted'], 
                     cv=10):
    _,__,f1_w,f1_m,r,p=cross_validate(model, X_train, y_train, cv=cv,
                       scoring=scoring).items()
    mean_f1_w = f1_w[1].mean(); sd_f1_w = f1_w[1].std()
    mean_f1_m = f1_m[1].mean(); sd_f1_m = f1_m[1].std()
    mean_r = r[1].mean(); sd_r = r[1].std()
    mean_p = p[1].mean(); sd_p = p[1].std()
    
    print("Classification report for classifier {}:\nparameters:\n".format(model) +
          "{}\nmean: {}, std deviation: {}\n\n".format(f1_w[0],mean_f1_w,sd_f1_w) +
          "{}\nmean: {}, std deviation: {}\n\n".format(f1_m[0],mean_f1_m,sd_f1_m) +
          "{}\nmean: {}, std deviation: {}\n\n".format(r[0],mean_r,sd_r) +
          "{}\nmean: {}, std deviation: {}\n\n".format(p[0],mean_p,sd_p))

In [46]:
def print_val_custom_score (model, X_train, y_train, 
                            scoring='f1_macro', 
                            cv=10):
    print("Classification report for classifier {}:\nparameters:\n".format(model))
    
    myReturn=cross_validate(model,X_train,y_train,cv=cv,scoring=scoring)
    _,__,metric_score=myReturn
    metric_score=myReturn[metric_score]
    mean = np.mean(metric_score)
    sd = np.std(metric_score)
    print("{}\nmean: {}, std deviation: {}\n\n".format(scoring,mean,sd))

In [47]:
def print_metrics(clf, X, y, prediction):
    print(f"Classification report for classifier {clf}:\n"
          f"{metrics.classification_report(y, prediction)}\n")
    disp = metrics.plot_confusion_matrix(clf, X, y)
    disp.figure_.suptitle("Confusion Matrix")

In [11]:
data=pd.read_csv('heart_pred.csv')
# data.describe()

Retirando valores nulos e NaN

In [12]:
data=data.dropna()
# data_columns=list(data.columns.values.tolist())
data=data[data.notnull()]
data=data.reset_index()
data=data.sample(frac=0.1)
data.describe()

,index,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
count,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000
mean,2171.243169,0.475410,49.546448,2.024590,0.489071,8.953552,0.021858,0.005464,0.295082,0.038251,238.008197,131.769126,83.233607,25.654809,76.038251,85.428962,0.163934
std,1230.105009,0.500079,8.330947,1.071139,0.500565,12.152029,0.146420,0.073821,0.456704,0.192065,42.957225,19.795235,11.110007,3.997785,11.510503,29.555041,0.370723
min,18.000000,0.000000,34.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,135.000000,90.000000,59.000000,16.730000,45.000000,53.000000,0.000000
25%,1055.500000,0.000000,43.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,210.000000,119.000000,75.125000,23.007500,69.000000,72.000000,0.000000
50%,2209.500000,0.000000,49.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,237.000000,128.750000,82.000000,25.495000,75.000000,78.000000,0.000000
75%,3248.500000,1.000000,56.000000,3.000000,1.000000,20.000000,0.000000,0.000000,1.000000,0.000000,263.000000,142.875000,89.875000,28.037500,85.000000,90.000000,0.000000
max,4222.000000,1.000000,69.000000,4.000000,1.000000,70.000000,1.000000,1.000000,1.000000,1.000000,410.000000,205.000000,124.000000,42.000000,122.000000,370.000000,1.000000


In [13]:
y=data["TenYearCHD"]
X=data.drop('TenYearCHD',axis=1)

In [14]:
# X = TSNE(n_components=2).fit_transform(X)
pca = PCA(n_components=3)
pca.fit(X)

PCA(n_components=3)

In [15]:
X_reduced=pca.transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.20, random_state = 200)

In [16]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [9]:
# print data distribution after t-SNE reduction
print_dim_reduced_data(X_train, y_train)
# print_2d_data(X_train, y_train)

NameError: name 'print_dim_reduced_data' is not defined

In [25]:
f_beta = make_scorer(fbeta_score, beta=2, average='weighted')

In [34]:
# Criacao de grid para a Random Forest Classifier
max_depth = [int(x) for x in np.linspace(5, 100, num = 5)]
max_depth.append(None)

grid_rfc = {
    'n_estimators': [int(x) for x in np.linspace(start = 100, stop = 600, num = 6)],
    'criterion': ['gini','entropy'],
    'max_features': ['auto', 'sqrt', 'log2', None],
    'max_depth': max_depth,
    'min_samples_split': [2, 4],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

In [35]:
clf_rfc = RandomForestClassifier()

In [36]:
#  Random search of parameters. Uses our shuffle selection cross validation, 
#  and search across "n_iter" different (shuffle) combinations, and use all available cores
rfc_search = RandomizedSearchCV(estimator = clf_rfc, param_distributions = grid_rfc, 
                               n_iter = 20, cv = 5, random_state=200, scoring=f_beta)

rfc_search.fit(X_train, y_train)
rfc_search.best_params_

{'n_estimators': 100,
 'min_samples_split': 4,
 'min_samples_leaf': 2,
 'max_features': 'log2',
 'max_depth': 5,
 'criterion': 'gini',
 'bootstrap': False}

In [37]:
rfc_search.best_score_

0.8130380130828172

In [51]:
best_clf_rfc = rfc_search.best_estimator_
print_val_scores(best_clf_rfc, X_train, y_train)

/home/ianloron00/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ianloron00/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ianloron00/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ianloron00/.local/lib/python3.8/site-packages/sklearn/metrics

Classification report for classifier RandomForestClassifier(bootstrap=False, max_depth=5, max_features='log2',
                       min_samples_leaf=2, min_samples_split=4):
parameters:
test_f1_weighted
mean: 0.7672547306325868, std deviation: 0.030913561754922676

test_f1_macro
mean: 0.46715705985961264, std deviation: 0.04298435965544746

test_recall_weighted
mean: 0.8287356321839081, std deviation: 0.026808974229173766

test_precision_weighted
mean: 0.7151261870643559, std deviation: 0.04074736498918468




/home/ianloron00/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [52]:
print_val_custom_score(best_clf_rfc, X_train, y_train,f_beta)

Classification report for classifier RandomForestClassifier(bootstrap=False, max_depth=5, max_features='log2',
                       min_samples_leaf=2, min_samples_split=4):
parameters:

make_scorer(fbeta_score, beta=2, average=weighted)
mean: 0.8019601050025875, std deviation: 0.02811101140265819




SVM implementarion

In [40]:
clf_svm = make_pipeline(StandardScaler(), SVC())
print(pprint(clf_svm.get_params() ))

{'memory': None,
 'standardscaler': StandardScaler(),
 'standardscaler__copy': True,
 'standardscaler__with_mean': True,
 'standardscaler__with_std': True,
 'steps': [('standardscaler', StandardScaler()), ('svc', SVC())],
 'svc': SVC(),
 'svc__C': 1.0,
 'svc__break_ties': False,
 'svc__cache_size': 200,
 'svc__class_weight': None,
 'svc__coef0': 0.0,
 'svc__decision_function_shape': 'ovr',
 'svc__degree': 3,
 'svc__gamma': 'scale',
 'svc__kernel': 'rbf',
 'svc__max_iter': -1,
 'svc__probability': False,
 'svc__random_state': None,
 'svc__shrinking': True,
 'svc__tol': 0.001,
 'svc__verbose': False,
 'verbose': False}
None


É assim que se imprime os parâmetros de cada modelo. Agora, precisa treinar para os demais

In [38]:
# https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
C = [float(x) for x in np.linspace(start = 0.1, stop = 100.0, num = 10).astype(float)]
class_weight=['balanced', None] #,{0:2,1:3},{0:5,1:1},{0:20,1:1},{0:200,1:3},{0:110,1:20}]
kernel = ['linear', 'poly', 'rbf','sigmoid']
gamma = ['scale', 'auto']
decision_function_shape = ['ovo','ovr']
shrinking = [True, False]

grid_svm = {'svc__C': C,
                   'svc__kernel': kernel,
                   'svc__gamma': gamma,
                   'svc__class_weight':class_weight,
                   'svc__decision_function_shape': decision_function_shape,
                   'svc__shrinking': shrinking}
# pprint(grid_svm)

In [41]:
svm_search = RandomizedSearchCV(estimator = clf_svm, param_distributions = grid_svm, 
                               n_iter = 20, cv = 5, random_state=200, scoring=f_beta)

svm_search.fit(X_train, y_train)
svm_search.best_params_

{'svc__shrinking': True,
 'svc__kernel': 'rbf',
 'svc__gamma': 'auto',
 'svc__decision_function_shape': 'ovo',
 'svc__class_weight': None,
 'svc__C': 44.50000000000001}

In [48]:
clf_svm = svm_search.best_estimator_
svm_search.best_score_
print_val_custom_score(best_clf_svm,X_train,y_train,f_beta)

Classification report for classifier Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc',
                 SVC(C=44.50000000000001, decision_function_shape='ovo',
                     gamma='auto'))]):
parameters:

make_scorer(fbeta_score, beta=2, average=weighted)
mean: 0.8032301657604393, std deviation: 0.03415055609091998




In [49]:
print_val_scores(best_clf_svm,X_train,y_train)

Classification report for classifier Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc',
                 SVC(C=44.50000000000001, decision_function_shape='ovo',
                     gamma='auto'))]):
parameters:
test_f1_weighted
mean: 0.7687812932419819, std deviation: 0.03512788348917426

test_f1_macro
mean: 0.4694712284057457, std deviation: 0.05429699616844974

test_recall_weighted
mean: 0.8286206896551723, std deviation: 0.035001651434153506

test_precision_weighted
mean: 0.7266741218881527, std deviation: 0.059973731929896065




/home/ianloron00/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ianloron00/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ianloron00/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ianloron00/.local/lib/python3.8/site-packages/sklearn/metrics

In [28]:
clf_lr = LogisticRegression(random_state=200)
clf_rfc = RandomForestClassifier(random_state=200)
clf_nb = GaussianNB()

print('5 fold cross-validation')

labels = ['Logistic Regression','Random Forest Classsifier', 'Naive Bayes']
for clf, label in zip([clf_lr, clf_rfr,clf_nb], labels):

    scores = cross_val_score(clf, X_train, y_train,
                             cv=5, scoring=f_beta)

    print("f2 Validation: %0.3f (+/- %0.3f) [%s]" % (np.mean(scores), np.std(scores), label))

5 fold cross-validation
F2 Validation: 0.812 (+/- 0.018) [Logistic Regression]
F2 Validation: 0.809 (+/- 0.010) [Random Forest Classsifier]
F2 Validation: 0.813 (+/- 0.027) [Naive Bayes]


In [30]:
voting_clf_hard = VotingClassifier(estimators = [(labels[0], clf_lr),
                                                 (labels[1], clf_rfc),
                                                 (labels[2], clf_nb)],
                                  voting='hard')

In [31]:
voting_clf_soft = VotingClassifier(estimators = [(labels[0], clf_lr),
                                                 (labels[1], clf_rfc),
                                                 (labels[2], clf_nb)],
                                  voting='soft')

In [32]:
print('5 fold cross-validation')

labels = ['Logistic Regression','Random Forest Classifier', 'Naive Bayes', 'Voting Classifier - Hard', 
          'Voting Classifier - Soft']
for clf, label in zip([clf_lr, clf_rfc,clf_nb, voting_clf_hard, voting_clf_soft], labels):

    scores = cross_val_score(clf, X_train, y_train,
                             cv=5, scoring=f_beta)

    print("f2 Validation: %0.3f (+/- %0.3f) [%s]" % (np.mean(scores), np.std(scores), label))

5 fold cross-validation
f2 Validation: 0.812 (+/- 0.018) [Logistic Regression]
f2 Validation: 0.809 (+/- 0.010) [Random Forest Classsifier]
f2 Validation: 0.813 (+/- 0.027) [Naive Bayes]
f2 Validation: 0.812 (+/- 0.018) [Hard Voting]
f2 Validation: 0.804 (+/- 0.011) [Soft Voting]
